In [ ]:
from tools_definition import CustomShoppingSearchTool,CustomProductSearchTool,CustomScenarioVectorSearchTool
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import AzureChatOpenAI
import os

In [4]:
llm = AzureChatOpenAI(
    openai_api_base="https://openai-smart.openai.azure.com/",
    openai_api_version="2023-07-01-preview",
    deployment_name="smart",
    openai_api_key='4282343cdf384587a875631d9f8428e9',
    openai_api_type="azure",
    temperature=0
)

In [5]:
os.environ['SERPAPI_API_KEY'] = '96af9cb36955c3e23ba62c7776c2decb01505e3bc17247386a95db6779db485c'

In [7]:
agent = initialize_agent(
    [CustomShoppingSearchTool(),CustomProductSearchTool(),CustomScenarioVectorSearchTool()], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [8]:
agent.run(
    "I want to buy an iphone 14"
)



> Entering new AgentExecutor chain...
I need to find information about the availability and pricing of the iPhone 14.
Action: custom_vector_search
Action Input: "iPhone 14 availability and pricing"

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Unable to compare versions for numpy>=1.17: need=1.17 found=None. This is unusual. Consider reinstalling numpy.

In [18]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.utilities import GoogleSearchAPIWrapper

In [21]:
tools = [CustomShoppingSearchTool(),CustomProductSearchSchema()]

In [22]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [24]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [26]:
agent_chain.run(input="Hello my name is Gautam")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: This is just an introduction, no action needed.`

In [27]:
agent_chain.run(input="What is my name")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: I don't have access to personal information
Action: None`

In [28]:
!pip install langchain
!pip install langchain_experimental
!pip install openai
!pip install google-search-results

  Obtaining dependency information for langchain_experimental from https://files.pythonhosted.org/packages/98/18/7a30314eab8e74aa82416c6629dff8ace4226c61ff9c78ba183fcd5f29ee/langchain_experimental-0.0.31-py3-none-any.whl.metadata
  Obtaining dependency information for langchain>=0.0.308 from https://files.pythonhosted.org/packages/cb/58/27830eb8bdcfabc2ec9ce2fef4e9d4fda5ce7c886f6b5be18d2ee365e603/langchain-0.0.316-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.43 from https://files.pythonhosted.org/packages/ba/eb/4212c401170f85ff4a72f6cd8b91f9e6b0cbf17d317be40365e5bcdfbd58/langsmith-0.0.45-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpointer>=1.9 from https://files.pythonhosted.org/packages/12/f6/0232cc0c617e1

In [29]:
import os
os.environ['OPENAI_API_KEY'] = str("4282343cdf384587a875631d9f8428e9")
os.environ["SERPAPI_API_KEY"] = str("96af9cb36955c3e23ba62c7776c2decb01505e3bc17247386a95db6779db485c")
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

ModuleNotFoundError: No module named 'langchain.schema.language_model'; 'langchain.schema' is not a package